# <span style="font-family:Courier New; color:#CCCCCC">**CADEC Named Entity Recognition CRF**</span>

## <span style="font-family:Courier New; color:#336666">**Load Data and Imports**</span>

In [57]:
from preprocessing import convert_BIO
from NER_evaluation import *
from feature_getter import Feature_getter
import pycrfsuite
from collections import Counter
import pandas as pd

In [55]:
def decode_cadec(file_path):

    ent_map = {1: 'ADR', 2: 'Di', 3: 'Dr', 4: 'S', 5: 'F'}
    data = list()
    with open(file_path, "r") as file:
        sent = list()
        for line in file:
            tok = line.split()
            if tok == []:
                "add previous sentence tokens"
                data.append(sent)
                sent = list()
            else:
                for i, label in enumerate(tok):
                    if i > 0 and label != 'O':
                        sent.append((tok[0], '', label[0] + '-' + ent_map[i]))
                        break
                    if i == len(tok) - 1:
                        sent.append((tok[0], '', 'O'))
    return data

train_data = decode_cadec('data/train.conll')
test_data = decode_cadec('data/test.conll')

In [58]:
train_BIO = convert_BIO(train_data)
test_BIO = convert_BIO(test_data)

X_test_BIO = [[word[0] for word in sent] for sent in test_BIO]
y_test_BIO = [[word[1] for word in sent] for sent in test_BIO]

## <span style="font-family:Courier New; color:#336666">**Train Classifier**</span>